In [2]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from PIL import Image
import pickle

In [4]:
Categories = ['क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'श', 'ष', 'स', 'ह', 'क्ष', 'त्र', 'ज्ञ','del','nothing','space']

In [5]:
flat_data_arr=[] #input array
target_arr=[] #output array

In [ ]:
!mkdir data

In [6]:
datadir='./nsl-alphabet/' 

In [8]:
c=1
for i in Categories:
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        edges = cv.Canny(img_array,100,200) #canney edhe detecton
        cv.imwrite("./data/"+str(c)+".jpg",edges)
        c=c+1
#         img_resized=resize(img_array,(150,150,3))        
#         flat_data_arr.append(img_resized.flatten())        
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully')


In [ ]:
!ls ./data

In [9]:
for i in range(870):
        img_array=imread("./data/"+str(i+1)+".jpg")
#         edges = cv.Canny(img_array,100,200) #canney edhe detecton
#         cv.imwrite("./data/"+str(c)+".jpg",edges)
#         c=c+1
        img_resized=resize(img_array,(150,150,1))        
        flat_data_arr.append(img_resized.flatten())        
#         target_arr.append(Categories.index(i+1))

In [10]:
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x=df.iloc[:,:-1] #input data 
y=df.iloc[:,-1] #output data

In [ ]:
len(flat_data[0])

In [11]:
test1 = flat_data[0].reshape((150,150,1))

In [12]:
plt.subplot(122),plt.imshow(test1,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()

In [13]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
svc=svm.SVC(probability=True)
model=GridSearchCV(svc,param_grid)

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)

In [ ]:
# model.fit(x_train,y_train)
# print('The Model is trained well with the given images')

In [ ]:
# filename = './finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

In [15]:
filename = './finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [17]:
y_pred=model.predict(x_test)
print("The predicted Data is :")
print(y_pred)
print("The actual data is:")
print(np.array(y_test))
print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")

In [18]:
y_pred=loaded_model.predict(x_test)
print("The predicted Data is :")
print(y_pred)
print("The actual data is:")
print(np.array(y_test))
print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")

In [19]:
# import libraries
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt 
import seaborn as sns            # visualization tool

f,ax = plt.subplots(figsize=(10, 10))
sns.heatmap(confusion_matrix(y_test,y_pred), annot=True, linewidths=.1, fmt= '.0f',ax=ax)     # generating a heat map
plt.xlabel("Predicted Class")
plt.ylabel("Actual Class")
plt.show()